In [46]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [47]:

transform = transforms.Compose([
    transforms.Resize(224), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

# Load dataset (Dùng CIFAR-10 thay MNIST vì là ảnh màu)
train_dataset = datasets.CIFAR10(root='./data', train=True, 
                                 download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, 
                                download=True, transform=transform)


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Lấy số lượng lớp để dùng
num_classes = len(train_dataset.classes)
print(f"Đã tải dữ liệu, num_classes = {num_classes}")

# Thêm thiết lập 'device'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

100%|██████████| 170M/170M [00:14<00:00, 11.5MB/s] 


Đã tải dữ liệu, num_classes = 10
Sử dụng thiết bị: cuda


In [49]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

In [50]:
# Tải mô hình
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes) # num_classes = 10
# Chuyển model sang device
model = model.to(device)
print("Đã tải ResNet-18 và thay thế lớp 'fc'.")

c:\Users\chong\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\chong\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Đã tải ResNet-18 và thay thế lớp 'fc'.


In [51]:
import torch.optim as optim

In [52]:
# Loss function
criterion = nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
print("Đã khởi tạo Criterion và Optimizer (chỉ cho lớp fc).")

Đã khởi tạo Criterion và Optimizer (chỉ cho lớp fc).


In [53]:
def validate(model, dataloader, criterion, device):
    model.eval() # Dòng 4
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad(): 
        for data, labels in dataloader: 
            data = data.to(device)
            labels = labels.to(device)

            outputs = model(data)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted_labels = torch.max(outputs.data, 1)
            total_samples += labels.size(0) 
            correct_predictions += (predicted_labels == labels).sum().item()

    avg_loss = total_loss / len(dataloader) 
    accuracy = 100 * correct_predictions / total_samples 
    
    model.train() # đặt lại model về train mode
    
    return avg_loss, accuracy

In [54]:
# Write the Training Loop
num_epochs = 10 # Dòng 3

print("Bắt đầu huấn luyện...")

for epoch in range(num_epochs):
    model.train() 
    
    for batch_idx, (data, labels) in enumerate(train_dataloader): 
        data = data.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Print training statistics 
        if (batch_idx + 1) % 100 == 0: # In mỗi 100 batch
            print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')
            print(f'Batch[{batch_idx+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}')
    # Validation 
    val_loss, val_accuracy = validate(model, test_dataloader, criterion, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

print("Training finished!")

Bắt đầu huấn luyện...
Epoch [1/10], Batch [100/782], Loss: 1.0029
Batch[100/782], Loss: 1.0029
Epoch [1/10], Batch [200/782], Loss: 0.7714
Batch[200/782], Loss: 0.7714
Epoch [1/10], Batch [300/782], Loss: 0.6760
Batch[300/782], Loss: 0.6760
Epoch [1/10], Batch [400/782], Loss: 0.6066
Batch[400/782], Loss: 0.6066
Epoch [1/10], Batch [500/782], Loss: 0.8332
Batch[500/782], Loss: 0.8332
Epoch [1/10], Batch [600/782], Loss: 0.5744
Batch[600/782], Loss: 0.5744
Epoch [1/10], Batch [700/782], Loss: 0.7651
Batch[700/782], Loss: 0.7651
Epoch [1/10], Validation Loss: 0.6376, Validation Accuracy: 78.21%
Epoch [2/10], Batch [100/782], Loss: 0.5946
Batch[100/782], Loss: 0.5946
Epoch [2/10], Batch [200/782], Loss: 0.8155
Batch[200/782], Loss: 0.8155
Epoch [2/10], Batch [300/782], Loss: 0.7133
Batch[300/782], Loss: 0.7133
Epoch [2/10], Batch [400/782], Loss: 0.5116
Batch[400/782], Loss: 0.5116
Epoch [2/10], Batch [500/782], Loss: 0.6463
Batch[500/782], Loss: 0.6463
Epoch [2/10], Batch [600/782], Loss

In [55]:
# Save and Load Your Model
torch.save(model.state_dict(), 'model.pth')
print("Đã lưu model state dict vào 'model.pth'")

Đã lưu model state dict vào 'model.pth'


In [56]:
loaded_model = models.resnet18(pretrained=False) # Không cần pre-trained
num_ftrs = loaded_model.fc.in_features
loaded_model.fc = nn.Linear(num_ftrs, num_classes) # num_classes = 10
loaded_model.to(device)
loaded_model.load_state_dict(torch.load('model.pth'))
loaded_model.eval()

print("Đã tải model đã lưu.")

# Kiểm tra nhanh
test_loss, test_acc = validate(loaded_model, test_dataloader, criterion, device)
print(f'Kiểm tra mô hình đã tải - Loss: {test_loss:.4f}, Accuracy: {test_acc:.2f}%')

c:\Users\chong\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\chong\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Đã tải model đã lưu.
Kiểm tra mô hình đã tải - Loss: 0.5589, Accuracy: 80.88%
